In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import malaya_speech.config
from malaya_speech.train.model import squeezeformer, transducer
import malaya_speech
import tensorflow as tf
import numpy as np

2022-08-17 05:42:11.040774: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/husein/tf-nvidia/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
subwords = malaya_speech.subword.load('transducer.subword.subwords')

In [4]:
featurizer = malaya_speech.tf_featurization.STTFeaturizer(
    normalize_per_feature = True
)

In [5]:
X = tf.compat.v1.placeholder(tf.float32, [None, None], name = 'X_placeholder')
X_len = tf.compat.v1.placeholder(tf.int32, [None], name = 'X_len_placeholder')

In [6]:
batch_size = tf.shape(X)[0]
features = tf.TensorArray(dtype = tf.float32, size = batch_size, dynamic_size = True, infer_shape = False)
features_len = tf.TensorArray(dtype = tf.int32, size = batch_size)

init_state = (0, features, features_len)

def condition(i, features, features_len):
    return i < batch_size

def body(i, features, features_len):
    f = featurizer(X[i, :X_len[i]])
    f_len = tf.shape(f)[0]
    return i + 1, features.write(i, f), features_len.write(i, f_len)

_, features, features_len = tf.while_loop(condition, body, init_state)
features_len = features_len.stack()
padded_features = tf.TensorArray(dtype = tf.float32, size = batch_size)
padded_lens = tf.TensorArray(dtype = tf.int32, size = batch_size)
maxlen = tf.reduce_max(features_len)

init_state = (0, padded_features, padded_lens)

def condition(i, padded_features, padded_lens):
    return i < batch_size

def body(i, padded_features, padded_lens):
    f = features.read(i)
    len_f = tf.shape(f)[0]
    f = tf.pad(f, [[0, maxlen - tf.shape(f)[0]], [0,0]])
    return i + 1, padded_features.write(i, f), padded_lens.write(i, len_f)

_, padded_features, padded_lens = tf.while_loop(condition, body, init_state)
padded_features = padded_features.stack()
padded_lens = padded_lens.stack()
padded_lens.set_shape((None,))
padded_features.set_shape((None, None, 80))
padded_features = tf.expand_dims(padded_features, -1)
padded_features, padded_lens

(<tf.Tensor 'ExpandDims:0' shape=(?, ?, 80, 1) dtype=float32>,
 <tf.Tensor 'TensorArrayStack_2/TensorArrayGatherV3:0' shape=(?,) dtype=int32>)

In [7]:
padded_features = tf.identity(padded_features, name = 'padded_features')
padded_lens = tf.identity(padded_lens, name = 'padded_lens')

In [8]:
config = malaya_speech.config.squeezeformer_m_encoder_config
config['encoder_dropout'] = 0.0
model = squeezeformer.Model(**config)


INFO:tensorflow:Subsampling with DS conv
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']


In [9]:
decoder_config = malaya_speech.config.conformer_base_decoder_config
decoder_config['embed_dropout'] = 0.0
transducer_model = transducer.rnn.Model(
    model, vocabulary_size = subwords.vocab_size, **decoder_config
)

In [10]:
p = tf.compat.v1.placeholder(tf.int32, [None, None])
z = tf.zeros((tf.shape(p)[0], 1),dtype=tf.int32)
c = tf.concat([z, p], axis = 1)
p_len = tf.compat.v1.placeholder(tf.int32, [None])
c

<tf.Tensor 'concat:0' shape=(?, ?) dtype=int32>

In [11]:
training = True

In [12]:
logits = transducer_model([padded_features, padded_lens, c, p_len], training = training)
logits

<tf.Tensor 'transducer/transducer_joint/transducer_joint_vocab/BiasAdd:0' shape=(?, ?, ?, 1030) dtype=float32>

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

2022-08-17 05:42:27.690477: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-08-17 05:42:27.731978: E tensorflow/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-17 05:42:27.732007: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: husein-MS-7D31
2022-08-17 05:42:27.732010: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: husein-MS-7D31
2022-08-17 05:42:27.732082: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.141.3
2022-08-17 05:42:27.732097: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.141.3
2022-08-17 05:42:27.732100: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.141.3
2022-08-17 05:42:27.737849: I tensorflow/core/platf

In [14]:
var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
saver = tf.train.Saver(var_list = var_list)
saver.restore(sess, 'asr-m-squeezeformer-transducer/model.ckpt-800000')




INFO:tensorflow:Restoring parameters from asr-m-squeezeformer-transducer/model.ckpt-800000


In [15]:
padded_features, padded_lens

(<tf.Tensor 'padded_features:0' shape=(?, ?, 80, 1) dtype=float32>,
 <tf.Tensor 'padded_lens:0' shape=(?,) dtype=int32>)

In [16]:
decoded = transducer_model.greedy_decoder(padded_features, padded_lens, training = training)
decoded = tf.identity(decoded[0], name = 'greedy_decoder')
decoded

<tf.Tensor 'greedy_decoder:0' shape=(?, ?) dtype=int32>

In [17]:
decoded

<tf.Tensor 'greedy_decoder:0' shape=(?, ?) dtype=int32>

In [18]:
encoded = transducer_model.encoder(padded_features, padded_lens, training = training)
encoded

<tf.Tensor 'conformer_1/conformer_encoder/conformer_encoder_block_19/conformer_encoder_block_19_layer3/conformer_encoder_block_19_layer3_ln/batchnorm/add_1:0' shape=(?, ?, 324) dtype=float32>

In [19]:
encoded = tf.identity(encoded, name = 'encoded')

In [20]:
encoded_placeholder = tf.placeholder(tf.float32, [config['encoder_dmodel']], name = 'encoded_placeholder')
predicted_placeholder = tf.placeholder(tf.int32, None, name = 'predicted_placeholder')
t = transducer_model.predict_net.get_initial_state().shape
states_placeholder = tf.placeholder(tf.float32, [int(i) for i in t], name = 'states_placeholder')

ytu, new_states = transducer_model.decoder_inference(
    encoded=encoded_placeholder,
    predicted=predicted_placeholder,
    states=states_placeholder,
    training = training
)

ytu = tf.identity(ytu, name = 'ytu')
new_states = tf.identity(new_states, name = 'new_states')
ytu, new_states

(<tf.Tensor 'ytu:0' shape=(1030,) dtype=float32>,
 <tf.Tensor 'new_states:0' shape=(1, 2, 1, 640) dtype=float32>)

In [21]:
initial_states = transducer_model.predict_net.get_initial_state()
initial_states = tf.identity(initial_states, name = 'initial_states')

In [22]:
files = [
    'speech/record/savewav_2020-11-26_22-36-06_294832.wav',
    'speech/record/savewav_2020-11-26_22-40-56_929661.wav',
    'speech/record/675.wav',
    'speech/record/664.wav',
    'speech/example-speaker/husein-zolkepli.wav',
    'speech/example-speaker/mas-aisyah.wav',
    'speech/example-speaker/khalil-nooh.wav',
    'speech/example-speaker/shafiqah-idayu.wav',
    'speech/khutbah/wadi-annuar.wav',
]

front_pad = 200
back_pad = 2000
inputs = [malaya_speech.load(f)[0] for f in files]
padded, lens = malaya_speech.padding.sequence_1d(inputs, return_len = True)
back = np.zeros(shape = (len(inputs), back_pad))
front = np.zeros(shape = (len(inputs), front_pad))
padded = np.concatenate([front, padded, back], axis = -1)
lens = [l + front_pad + back_pad for l in lens]

In [23]:
import collections
import numpy as np
import tensorflow as tf

BeamHypothesis = collections.namedtuple(
    'BeamHypothesis', ('score', 'prediction', 'states')
)


def transducer(
    enc,
    total,
    initial_states,
    encoded_placeholder,
    predicted_placeholder,
    states_placeholder,
    ytu,
    new_states,
    sess,
    beam_width = 10,
    norm_score = True,
):
    kept_hyps = [
        BeamHypothesis(score = 0.0, prediction = [0], states = initial_states)
    ]
    B = kept_hyps
    for i in range(total):
        A = B
        B = []
        while True:
            y_hat = max(A, key = lambda x: x.score)
            A.remove(y_hat)
            ytu_, new_states_ = sess.run(
                [ytu, new_states],
                feed_dict = {
                    encoded_placeholder: enc[i],
                    predicted_placeholder: y_hat.prediction[-1],
                    states_placeholder: y_hat.states,
                },
            )
            for k in range(ytu_.shape[0]):
                beam_hyp = BeamHypothesis(
                    score = (y_hat.score + float(ytu_[k])),
                    prediction = y_hat.prediction,
                    states = y_hat.states,
                )
                if k == 0:
                    B.append(beam_hyp)
                else:
                    beam_hyp = BeamHypothesis(
                        score = beam_hyp.score,
                        prediction = (beam_hyp.prediction + [int(k)]),
                        states = new_states_,
                    )
                    A.append(beam_hyp)
            if len(B) > beam_width:
                break
    if norm_score:
        kept_hyps = sorted(
            B, key = lambda x: x.score / len(x.prediction), reverse = True
        )[:beam_width]
    else:
        kept_hyps = sorted(B, key = lambda x: x.score, reverse = True)[
            :beam_width
        ]
    return kept_hyps[0].prediction

In [24]:
%%time

r = sess.run(decoded, feed_dict = {X: padded, X_len: lens})
for row in r:
    print(malaya_speech.subword.decode(subwords, row[row > 0]))

2022-08-17 05:42:51.966770: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 236079360 exceeds 10% of system memory.
2022-08-17 05:42:52.081442: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 242459568 exceeds 10% of system memory.
2022-08-17 05:42:52.154178: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 59019840 exceeds 10% of system memory.
2022-08-17 05:42:52.178204: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 59019840 exceeds 10% of system memory.
2022-08-17 05:42:52.644016: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 376257312 exceeds 10% of system memory.


helo nama saya mesin saya tak suka mandi ketat saya masam
helo nama saya husin saya suka mandi saya mandi setiap hari
ini dan melalui kenyataan mesej itu mastura menegaskan
pilihan tepat apabila dia kini lebih berani dan
testing nama saya husin bin zulkari
sebut perkataan uncle
tolong sebut antikata
nama saya syafiqa hidayah
jadi dalam perjalanan ini dunia yang susah ini ketika nabi mengajar muaz bin jabal tadi ni allahumma in
CPU times: user 28.7 s, sys: 3.93 s, total: 32.7 s
Wall time: 5.57 s


In [25]:
%%time

encoded_, padded_lens_  = sess.run([encoded, padded_lens], feed_dict = {X: padded, X_len: lens})
padded_lens_ = padded_lens_ // model.encoder.conv_subsampling.time_reduction_factor
s = sess.run(initial_states)

for i in range(len(encoded_)):
    r = transducer(
        enc = encoded_[i],
        total = padded_lens_[i],
        initial_states = s,
        encoded_placeholder = encoded_placeholder,
        predicted_placeholder = predicted_placeholder,
        states_placeholder = states_placeholder,
        ytu = ytu,
        new_states = new_states,
        sess = sess,
        beam_width = 1,
    )

    print(malaya_speech.subword.decode(subwords, r))

helo nama saya mesin saya tak suka mandi ketat saya masam
helo nama saya husin saya suka mandi saya mandi setiap hari
ini dan melalui kenyataan mesej itu mastura menegaskan
pilihan tepat apabila dia kini lebih berani dan
testing nama saya husin bin zulkapli
sebut perkataan uncle
tolong sebut antikata
nama saya syafiqa hidayah
jadi dalam perjalanan ini dunia yang susah ini ketika nabi mengajar muaz bin jabal tadi ni allahu
CPU times: user 39.6 s, sys: 2.02 s, total: 41.6 s
Wall time: 11.2 s


In [26]:
l = padded_lens // model.encoder.conv_subsampling.time_reduction_factor
encoded = transducer_model.encoder(padded_features, padded_lens, training = training)
g = transducer_model._perform_greedy(encoded[0], l[0],
                                tf.constant(0, dtype = tf.int32),
                                transducer_model.predict_net.get_initial_state())
g

Hypothesis(index=<tf.Tensor 'while_3/Exit_1:0' shape=() dtype=int32>, prediction=<tf.Tensor 'TensorArrayStack_3/TensorArrayGatherV3:0' shape=(?,) dtype=int32>, states=<tf.Tensor 'while_3/Exit_3:0' shape=(1, 2, 1, 640) dtype=float32>, alignment=<tf.Tensor 'TensorArrayStack_4/TensorArrayGatherV3:0' shape=(?, 1030) dtype=float32>)

In [27]:
indices = g.prediction
minus_one = -1 * tf.ones_like(indices, dtype=tf.int32)
blank_like = 0 * tf.ones_like(indices, dtype=tf.int32)
indices = tf.where(indices == minus_one, blank_like, indices)
num_samples = tf.cast(X_len[0], dtype=tf.float32)
total_time_reduction_factor = featurizer.frame_step
stime = tf.range(0, num_samples, delta=total_time_reduction_factor, dtype=tf.float32)
stime /= tf.cast(featurizer.sample_rate, dtype=tf.float32)
stime = stime[::tf.shape(stime)[0] // tf.shape(indices)[0]]
stime.set_shape((None,))
non_blank = tf.where(tf.not_equal(indices, 0))
non_blank_transcript = tf.gather_nd(indices, non_blank)
non_blank_stime = tf.gather_nd(stime, non_blank)
non_blank_transcript = tf.identity(non_blank_transcript, name = 'non_blank_transcript')
non_blank_stime = tf.identity(non_blank_stime, name = 'non_blank_stime')

In [28]:
%%time

r = sess.run([non_blank_transcript, non_blank_stime], feed_dict = {X: padded, X_len: lens})

CPU times: user 13.1 s, sys: 1.29 s, total: 14.4 s
Wall time: 4.42 s


In [29]:
words, indices = [], []
for no, ids in enumerate(r[0]):
    w = subwords._id_to_subword(ids - 1)
    if type(w) == bytes:
        w = w.decode()
    words.extend([w, None])
    indices.extend([no, None])

In [30]:
import six
from malaya_speech.utils import text_encoder

def _trim_underscore_and_tell(token):
    if token.endswith('_'):
        return token[:-1], True
    return token, False

def decode(ids):
    ids = text_encoder.pad_decr(ids)
    subword_ids = ids
    del ids

    subwords_ = []
    prev_bytes = []
    prev_ids = []
    ids = []

    def consume_prev_bytes():
        if prev_bytes:
            subwords_.extend(prev_bytes)
            ids.extend(prev_ids)
        return [], []

    for no, subword_id in enumerate(subword_ids):
        subword = subwords._id_to_subword(subword_id)
        if isinstance(subword, six.binary_type):
            # Byte-encoded
            prev_bytes.append(subword.decode('utf-8', 'replace'))
            if subword == b' ':
                prev_ids.append(None)
            else:
                prev_ids.append(no)
        else:
            # If there were bytes previously, convert to unicode.
            prev_bytes, prev_ids = consume_prev_bytes()
            trimmed, add_space = _trim_underscore_and_tell(subword)
            ids.append(no)
            subwords_.append(trimmed)
            if add_space:
                subwords_.append(' ')
                ids.append(None)
    prev_bytes = consume_prev_bytes()

    return subwords_, ids

words, indices = decode(r[0])

In [31]:
def combined_indices(subwords, ids, l, reduction_factor = 160, sample_rate = 16000):
    result, temp_l, temp_r = [], [], []
    for i in range(len(subwords)):
        if ids[i] is not None:
            temp_l.append(subwords[i])
            temp_r.append(l[ids[i]])
        else:
            data = {'text': ''.join(temp_l), 
                    'start': round(temp_r[0],4), 
                    'end': round(temp_r[-1] + (reduction_factor / sample_rate), 4)}
            result.append(data)
            temp_l, temp_r = [], []
    
    if len(temp_l):
        data = {'text': ''.join(temp_l), 
                'start': round(temp_r[0],4), 
                'end': round(temp_r[-1] + (reduction_factor / sample_rate), 4)}
        result.append(data)
    
    return result

In [32]:
combined_indices(words, indices, r[1])

[{'text': 'helo', 'start': 0.04, 'end': 0.17},
 {'text': 'nama', 'start': 0.8, 'end': 1.01},
 {'text': 'saya', 'start': 1.08, 'end': 1.33},
 {'text': 'mesin', 'start': 1.48, 'end': 1.77},
 {'text': 'saya', 'start': 2.68, 'end': 2.77},
 {'text': 'tak', 'start': 2.84, 'end': 2.85},
 {'text': 'suka', 'start': 2.96, 'end': 3.17},
 {'text': 'mandi', 'start': 3.2, 'end': 3.41},
 {'text': 'ketat', 'start': 4.6, 'end': 4.77},
 {'text': 'saya', 'start': 4.88, 'end': 5.09},
 {'text': 'masam', 'start': 5.32, 'end': 5.61}]

In [33]:
saver = tf.train.Saver()
saver.save(sess, 'output-m-squeezeformer/model.ckpt')

'output-m-squeezeformer/model.ckpt'

In [34]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'gather' in n.op.lower()
        or 'placeholder' in n.name
        or 'encoded' in n.name
        or 'decoder' in n.name
        or 'ytu' in n.name
        or 'new_states' in n.name
        or 'padded_' in n.name
        or 'initial_states' in n.name
        or 'non_blank' in n.name)
        and 'adam' not in n.name
        and 'global_step' not in n.name
        and 'Assign' not in n.name
        and 'ReadVariableOp' not in n.name
        and 'Gather' not in n.name
    ]
)
strings.split(',')

['X_placeholder',
 'X_len_placeholder',
 'padded_features',
 'padded_lens',
 'conformer_encoder_block_0_layer1_mhsa_scale',
 'conformer_encoder_block_0_layer1_mhsa_bias',
 'conformer_encoder_block_0_layer1_ff_scale',
 'conformer_encoder_block_0_layer1_ff_bias',
 'conformer_encoder_block_0_layer3_conv_scale',
 'conformer_encoder_block_0_layer3_conv_bias',
 'conformer_encoder_block_0_layer3_ff_scale',
 'conformer_encoder_block_0_layer3_ff_bias',
 'conformer_encoder_block_1_layer1_mhsa_scale',
 'conformer_encoder_block_1_layer1_mhsa_bias',
 'conformer_encoder_block_1_layer1_ff_scale',
 'conformer_encoder_block_1_layer1_ff_bias',
 'conformer_encoder_block_1_layer3_conv_scale',
 'conformer_encoder_block_1_layer3_conv_bias',
 'conformer_encoder_block_1_layer3_ff_scale',
 'conformer_encoder_block_1_layer3_ff_bias',
 'conformer_encoder_block_2_layer1_mhsa_scale',
 'conformer_encoder_block_2_layer1_mhsa_bias',
 'conformer_encoder_block_2_layer1_ff_scale',
 'conformer_encoder_block_2_layer1_ff_b

In [35]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [36]:
freeze_graph('output-m-squeezeformer', strings)



INFO:tensorflow:Restoring parameters from output-m-squeezeformer/model.ckpt
INFO:tensorflow:Froze 887 variables.
INFO:tensorflow:Converted 887 variables to const ops.

40496 ops in the final graph.


In [37]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
                
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
        
    return graph

In [38]:
g = load_graph('output-m-squeezeformer/frozen_model.pb')

In [39]:
input_nodes = [
    'X_placeholder',
    'X_len_placeholder',
    'encoded_placeholder',
    'predicted_placeholder',
    'states_placeholder',
]
output_nodes = [
    'greedy_decoder',
    'encoded',
    'ytu',
    'new_states',
    'padded_features',
    'padded_lens',
    'initial_states',
    'non_blank_transcript',
    'non_blank_stime'
]
inputs = {n: g.get_tensor_by_name(f'import/{n}:0') for n in input_nodes}
outputs = {n: g.get_tensor_by_name(f'import/{n}:0') for n in output_nodes}

In [40]:
test_sess = tf.Session(graph = g)

In [41]:
r = test_sess.run(outputs['greedy_decoder'], feed_dict = {inputs['X_placeholder']: padded, 
                                                          inputs['X_len_placeholder']: lens})

In [42]:
for row in r:
    print(malaya_speech.subword.decode(subwords, row[row > 0]))

helo nama saya mesin saya tak suka mandi ketat saya masam
helo nama saya husin saya suka mandi saya mandi setiap hari
ini dan melalui kenyataan mesej itu mastura menegaskan
pilihan tepat apabila dia kini lebih berani dan
testing nama saya husin bin zulkari
sebut perkataan uncle
tolong sebut antikata
nama saya syafiqa hidayah
jadi dalam perjalanan ini dunia yang susah ini ketika nabi mengajar muaz bin jabal tadi ni allahumma in


In [43]:
encoded_, padded_lens_, s  = test_sess.run([outputs['encoded'], outputs['padded_lens'], outputs['initial_states']], 
                                        feed_dict = {inputs['X_placeholder']: padded, 
                                                     inputs['X_len_placeholder']: lens})

padded_lens_ = padded_lens_ // model.encoder.conv_subsampling.time_reduction_factor

In [44]:
i = 0
r = transducer(
    enc = encoded_[i],
    total = padded_lens_[i],
    initial_states = s,
    encoded_placeholder = inputs['encoded_placeholder'],
    predicted_placeholder = inputs['predicted_placeholder'],
    states_placeholder = inputs['states_placeholder'],
    ytu = outputs['ytu'],
    new_states = outputs['new_states'],
    sess = test_sess,
    beam_width = 1,
)

malaya_speech.subword.decode(subwords, r)

'helo nama saya mesin saya tak suka mandi ketat saya masam'

In [45]:
from tensorflow.tools.graph_transforms import TransformGraph

In [46]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

input_nodes = [
    'X_placeholder',
    'X_len_placeholder',
    'encoded_placeholder',
    'predicted_placeholder',
    'states_placeholder',
]
output_nodes = [
    'greedy_decoder',
    'encoded',
    'ytu',
    'new_states',
    'padded_features',
    'padded_lens',
    'initial_states',
    'non_blank_transcript',
    'non_blank_stime'
]

pb = 'output-m-squeezeformer/frozen_model.pb'

input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())

transformed_graph_def = TransformGraph(input_graph_def, 
                                           input_nodes,
                                           output_nodes, transforms)
    
with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

2022-08-17 05:43:46.536364: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying add_default_attributes
2022-08-17 05:43:46.770995: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying remove_nodes
2022-08-17 05:43:46.987655: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for padded_features
2022-08-17 05:43:46.987692: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for padded_lens
2022-08-17 05:43:47.009995: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for greedy_decoder
2022-08-17 05:43:47.025194: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for encoded
2022-08-17 05:43:47.025579: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for ytu
2022-08-17 05:43:47.025599: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for new_states
2022-08-17 05:43:47.025637: I tensorflow/tools/gra

In [47]:
g = load_graph('output-m-squeezeformer/frozen_model.pb.quantized')

In [48]:
inputs = {n: g.get_tensor_by_name(f'import/{n}:0') for n in input_nodes}
outputs = {n: g.get_tensor_by_name(f'import/{n}:0') for n in output_nodes}
test_sess = tf.Session(graph = g)

In [49]:
r = test_sess.run(outputs['greedy_decoder'], feed_dict = {inputs['X_placeholder']: padded, 
                                                          inputs['X_len_placeholder']: lens})

In [50]:
for row in r:
    print(malaya_speech.subword.decode(subwords, row[row > 0]))

helo nama saya mesin saya tak suka mandi ketat saya masam
helo nama saya husin saya suka mandi saya mandi setiap hari
ini dan melalui kenyataan mesej itu mastura menegaskan
pilihan tepat apabila dia kini lebih berani dan
testing nama saya husin bin zulkari
sebut perkataan uncle
tolong sebut antikata
nama saya syafiqa hidayah
jadi dalam perjalanan ini dunia yang susah ini ketika nabi mengajar muaz bin jabal tadi ni allah mak


In [51]:
from malaya_boilerplate.huggingface import upload_dict

In [52]:
!tar -cvf output-m-squeezeformer.tar output-m-squeezeformer

output-m-squeezeformer/
output-m-squeezeformer/frozen_model.pb.quantized
output-m-squeezeformer/frozen_model.pb
output-m-squeezeformer/model.ckpt.meta
output-m-squeezeformer/checkpoint
output-m-squeezeformer/model.ckpt.index
output-m-squeezeformer/model.ckpt.data-00000-of-00001


In [53]:
files_mapping = {'output-m-squeezeformer.tar': 'output-m-squeezeformer.tar'}
upload_dict(model = 'pretrained-speech-to-text-transducer', files_mapping = files_mapping)

/home/husein/tf-nvidia/lib/python3.8/site-packages/huggingface_hub/hf_api.py:91: FutureWarning: `name` and `organization` input arguments are deprecated and will be removed in v0.10. Pass `repo_id` instead.
  warnings.warn(
409 Client Error: Conflict for url: https://huggingface.co/api/repos/create (Request ID: NLnucOY8QqKCbzlje_s6w) - You already created this model repo


In [54]:
files_mapping = {'output-m-squeezeformer/frozen_model.pb': 'model.pb'}
upload_dict(model = 'speech-to-text-transducer-m-squeezeformer', files_mapping = files_mapping)

In [55]:
files_mapping = {'output-m-squeezeformer/frozen_model.pb.quantized': 'model.pb'}
upload_dict(model = 'speech-to-text-transducer-m-squeezeformer-quantized', files_mapping = files_mapping)